# Funkcionalno programiranje

Amer Hasanović

## Iznimke

Generiranje iznimki u `F#` kodu:

* `failwith` - baca iznimku tipa `System.Exception`
* `failwithf` - baca formatiranu iznimku tipa `System.Exception`
* `invalidArg` - baca iznimku tipa `ArgumentException`
* `nullArg` - baca iznimku tipa `NullArgumentException`
* `invalidOp` - baca iznimku tipa `InvalidOperationException`

In [166]:
let foo a b = 
  if b = 0 then 
    failwith "Foobar"
  else 
    a / b

foo 5 0

Error: System.Exception: Foobar
   at FSI_0171.foo(Int32 a, Int32 b)
   at <StartupCode$FSI_0171>.$FSI_0171.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)


Bilo koje `.dotNet` iznimke mogu se generirati putem `raise`

In [167]:
let foo a b = 
  if b = 0 then 
    System.DivideByZeroException() |> raise
  else 
    a / b

foo 5 0

Error: System.DivideByZeroException: Attempted to divide by zero.
   at FSI_0172.foo(Int32 a, Int32 b)
   at <StartupCode$FSI_0172>.$FSI_0172.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

Da bi neki tip mogao biti korišten kao tip za iznimku mora biti u nasljednoj hijerarhiji od tipa `System.Exception`

## `F#` iznimke

`F#` pojednostavljena definicija novog tipa za iznimke je u formatu:

```fsharp
exception ImeTipa of PostojeciTip
```

In [168]:
exception DijeljenjeNulom of string

let foo a b = 
  if b = 0 then 
    DijeljenjeNulom "ouch" |> raise
  else 
    a / b

foo 5 0

Error: FSI_0173+DijeljenjeNulom: DijeljenjeNulom "ouch"
   at FSI_0173.foo(Int32 a, Int32 b)
   at <StartupCode$FSI_0173>.$FSI_0173.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

## Tretman iznimki

Bačena iznimka propagira po sekvenci ugnježdenih poziva funkcija na steku, sve dok u nekoj od funkcija ne bude uhvaćena putem `try` i tretirana putem `with`.

* Ukoliko nigdje ne bude tretirana iznimka, runtime terminira program uz odgovarajući ispis u terminalu.

`try` i `with` **moraju** rezultirati sa vrijednostima istog tipa.

`with` radi podudaranje uzoraka na bačenoj iznimnoj vrijednosti.

* ukoliko je bačena iznimka od nekog `F#` tipa, uzorak za podudranja je konstruktor od tog tipa.
* Uzorci `:?` i `as` se koriste za iznimke ostalih `.dotNet` tipova.

In [169]:
exception DijeljenjeNulom of string
exception VodaUKoljenu of int

let foo a b = 
  if b = 0 then 
    DijeljenjeNulom "ouch" |> raise
  else 
    a / b

try 
  foo 5 0
with 
 | DijeljenjeNulom s -> printfn "%s" s; 100
 | VodaUKoljenu x -> printfn "%d" x; -4
 | :? System.AccessViolationException as e -> printfn "%A" e; 22
 | _ -> 300

ouch


100

`try-finally` je varijanta `try` u kojoj se `finally` blok uvijek izvrši, pri čemu `finally` blok mora vraćati `()`
 
* `try-finally` se ne može direktno komponovati sa `with`

In [170]:
let bar x y =
  try 
    try 
      x / y
    finally 
      printfn "Uvijek se izvršim"
  with 
  | :? System.DivideByZeroException -> printfn "Dijeljenje sa nulom"; 22

bar 8 4 |> printfn "%d"
bar 5 0 |> printfn "%d"

Uvijek se izvršim
2
Uvijek se izvršim
Dijeljenje sa nulom
22


Umjesto iznimki u funkcionalnom kodu se preferiraju potpune funkcije koje vraćaju `Option` ili `Result`.

* Prilikom interakcije sa imperativnim kodom iz `dotNet` biblioteka preporučljivo je napraviti tzv wrapper funkciju koja normalizira potencijalno bačene iznimke.

In [171]:
let someLibFun x y = x / y

let trySomeLibFun x y =
   try
       someLibFun x y |> Some
   with
   | :? System.DivideByZeroException -> None 

trySomeLibFun 8 4 |> printfn "%A"
trySomeLibFun 8 0 |> printfn "%A"

Some 2
None


## `F#` kolekcije

`F#` definira tri ugrađena kontejnera koji ne podržavaju mutacije, i to:

* `list`
* `set`
* `map`

## `list` kontejner

In [172]:
let t1 : list<int> = 0::2::4::6::8::10::[]
let t2 : int list  = [0; 2; 4; 6; 8; 10]

In [173]:
let t3 = [0..2..10]

In [174]:
let t4 = [ for i in 0..10 do 
             if i % 2 = 0 then i 
         ]

In [175]:
let t5 = List.init 6 (fun i -> i*2) 

In [176]:
t2 = t1 && t3 = t2 && t4 = t1 && t5 = t3

True

In [177]:
let t6 = [ for i in -3..1 do 
             for j in 0..2..5 -> (i,j)
         ]
t6 |> printfn "%A"

[(-3, 0); (-3, 2); (-3, 4); (-2, 0); (-2, 2); (-2, 4); (-1, 0); (-1, 2); (-1, 4);
 (0, 0); (0, 2); (0, 4); (1, 0); (1, 2); (1, 4)]


In [178]:
let t7 = [ for i in -3..1 do 
             for j in 0..2..5 do
              (i,j)
         ]
t7 = t6 |> printfn "%A"

true


In [179]:
let foo l = 
  match l with
  | [] -> "Prazan"
  | [_; b] -> $"{b}"
  | a::_::c::[] -> $"{a} {c}"
  | _::xs -> $"{xs}"

In [180]:
[2..8] |> foo |> printfn "%A"
[1;3;4] |> foo |> printfn "%A"
[(3,1); (4,5)] |> foo |> printfn "%A"

"[3; 4; 5; ... ]"
"1 4"
"(4, 5)"


In [181]:
[2;3] @ [-4..-1] |> printfn "%A"

[2; 3; -4; -3; -2; -1]


In [182]:
let z = [2;3;8;-4;1;5]

In [183]:
z[4..] |> printfn "%A"
z[..3] |> printfn "%A"
z[2..4] |> printfn "%A"
z[2..100] |> printfn "%A"
z[100] |> printfn "%A"


[1; 5]
[2; 3; 8; -4]
[8; -4; 1]
[8; -4; 1; 5]


Error: System.ArgumentException: The index was outside the range of elements in the list. (Parameter 'n')
   at Microsoft.FSharp.Collections.PrivateListHelpers.nth[a](FSharpList`1 l, Int32 n) in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4088
   at Microsoft.FSharp.Collections.FSharpList`1.get_Item(Int32 index) in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4149
   at <StartupCode$FSI_0188>.$FSI_0188.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

In [184]:
let z = [2;3;8;-4;1;5;3;2;18]

In [185]:
z |> List.take 3 |> printfn "%A"

[2; 3; 8]


In [186]:
z |> List.skip 3 |> printfn "%A"

[-4; 1; 5; 3; 2; 18]


In [187]:
z |> List.pairwise |> printfn "%A"

[(2, 3); (3, 8); (8, -4); (-4, 1); (1, 5); (5, 3); (3, 2); (2, 18)]


In [188]:
let z = [2;3;8;-4;1;5;3;2;18;3;3]

In [189]:
z |> List.map ( fun e -> string e ) |> printfn "%A"

["2"; "3"; "8"; "-4"; "1"; "5"; "3"; "2"; "18"; "3"; "3"]


In [190]:
z |> List.mapi ( fun i e -> $"{i} -> {e}" ) |> printfn "%A"

["0 -> 2"; "1 -> 3"; "2 -> 8"; "3 -> -4"; "4 -> 1"; "5 -> 5"; "6 -> 3"; "7 -> 2";
 "8 -> 18"; "9 -> 3"; "10 -> 3"]


In [191]:
z |> List.fold ( fun a e -> e :: a) [] |> printfn "%A"

[3; 3; 18; 2; 3; 5; 1; -4; 8; 3; 2]


In [192]:
List.foldBack ( fun e a -> e :: a) z [] |> printfn "%A"

[2; 3; 8; -4; 1; 5; 3; 2; 18; 3; 3]


In [193]:
z |> List.reduce (+) |> printfn "%A"

44


In [194]:
z |> List.scan (+) 0 |> printfn "%A"

[0; 2; 5; 13; 9; 10; 15; 18; 20; 38; 41; 44]


In [195]:
z |> List.filter ( fun e -> e % 3 = 0) |> printfn "%A"

[3; 3; 18; 3; 3]


In [196]:
z |> List.choose ( fun e -> if e > 2 then Some e else None) |> printfn "%A"

[3; 8; 5; 3; 18; 3; 3]


In [197]:
z |> List.groupBy (id) |> printfn "%A"

[(2, [2; 2]); (3, [3; 3; 3; 3]); (8, [8]); (-4, [-4]); (1, [1]); (5, [5]);
 (18, [18])]


In [198]:
z |> List.countBy (id) |> printfn "%A"

[(2, 2); (3, 4); (8, 1); (-4, 1); (1, 1); (5, 1); (18, 1)]


In [199]:
z |> List.partition ( fun e -> e % 3 = 0) |> printfn "%A"

([3; 3; 18; 3; 3], [2; 8; -4; 1; 5; 2])


In [200]:
z |> List.contains 5 |> printfn "%A"

true


In [201]:
z |> List.exists ( fun e -> e % 2 = 0) |> printfn "%A"

true


In [202]:
z |> List.tryFind ( fun e -> e % 9 = 0) |> printfn "%A"

Some 18


In [203]:
z |> List.forall ( fun e -> e % 3 = 0) |> printfn "%A"

false


In [204]:
z |> List.sort |> printfn "%A"

[-4; 1; 2; 2; 3; 3; 3; 3; 5; 8; 18]


In [205]:
z |> List.sortBy (fun x -> -x) |> printfn "%A"

[18; 8; 5; 3; 3; 3; 3; 2; 2; 1; -4]


In [206]:
z |> List.sortWith (fun a b -> compare b a) |> printfn "%A"

[18; 8; 5; 3; 3; 3; 3; 2; 2; 1; -4]


In [207]:
z |> List.iter (fun e -> e |> printf "%d ")

2 3 8 -4 1 5 3 2 18 3 3 

In [208]:
List.replicate 4 "f" |> printfn "%A"

["f"; "f"; "f"; "f"]


In [209]:
[1..3] |> List.collect ( fun e -> [0..e]) |> printfn "%A"

[0; 1; 0; 1; 2; 0; 1; 2; 3]


In [210]:
[ for i in [1..3] do 
   for j in [0..i] -> j ] |> printfn "%A"

[0; 1; 0; 1; 2; 0; 1; 2; 3]


## `set` kontejner

In [211]:
let z = [2;3;8;-4;1;5;3;2;18;3;3] |> Set.ofList
let p = Set.empty |> Set.add 3 |> Set.add 100 |> Set.add 18
let t = set [3;3;4]

In [212]:
z |> printfn "%A"
p |> printfn "%A"
t |> printfn "%A"

set [-4; 1; 2; 3; 5; 8; 18]
set [3; 18; 100]
set [3; 4]


In [213]:
z - p |> printfn "%A"
p - z |> printfn "%A"
p + z |> printfn "%A"

set [-4; 1; 2; 5; 8]
set [100]
set [-4; 1; 2; 3; 5; 8; 18; 100]


In [214]:
Set.isSubset p z |> printfn "%A"

false


In [215]:
Set.isSubset (p |> Set.remove 100) z |> printfn "%A"

true


In [216]:
Set.isSuperset z (p |> Set.remove 100) |> printfn "%A"

true


In [217]:
z |> Set.map (fun e -> string e) |> printfn "%A"

set ["-4"; "1"; "18"; "2"; "3"; "5"; "8"]


## `map` kontejner

In [218]:
let z = ["foo",3; "bar",-1; "tar", 8] |> Map.ofList
let p = Map.empty |> Map.add "foo" 3 |> Map.add "bar" -1

In [219]:
z["foo"] |> printfn "%A"

3


In [220]:
z["foobar"] |> printfn "%A"

Error: System.Collections.Generic.KeyNotFoundException: The given key was not present in the dictionary.
   at Microsoft.FSharp.Collections.MapTreeModule.throwKeyNotFound[a]() in D:\a\_work\1\s\src\FSharp.Core\map.fs:line 187
   at Microsoft.FSharp.Collections.MapTreeModule.find[TKey,TValue](IComparer`1 comparer, TKey k, MapTree`2 m) in D:\a\_work\1\s\src\FSharp.Core\map.fs:line 196
   at Microsoft.FSharp.Collections.FSharpMap`2.get_Item(TKey key) in D:\a\_work\1\s\src\FSharp.Core\map.fs:line 777
   at <StartupCode$FSI_0225>.$FSI_0225.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

In [221]:
z |>  Map.tryFind("foobar") |> printfn "%A"

None


In [222]:
z |> Map.containsKey "foobar" |> printfn "%A"

false


In [223]:
z |> Map.keys |> printfn "%A"

seq ["bar"; "foo"; "tar"]


In [224]:
z |> Map.values |> printfn "%A"

seq [-1; 3; 8]


In [225]:
z |> Map.map (fun k e -> $"{k}->{e}") |> printfn "%A"

map [("bar", "bar->-1"); ("foo", "foo->3"); ("tar", "tar->8")]


## `Array` kontejner

In [226]:
let t1 : int array  = [|0; 2; 4; 6; 8; 10|]

In [227]:
let t2 = [|0..2..10|]

In [228]:
let t3 = [| for i in 0..10 do 
             if i % 2 = 0 then i 
         |]

In [229]:
let t4 = Array.init 6 (fun i -> i*2) 

In [230]:
t1 = t2 && t2 = t3 && t3 = t4 

True

In [231]:
let foo l = 
  match l with
  | [||] -> "Prazan"
  | [|_; b|] -> $"{b}"
  | [|a; b; c|] -> $"{a + b + c}"
  | x -> $"Svi: {x}"

In [232]:
[|0..2|] |> foo |> printfn "%A"
[|3|] |> foo |> printfn "%O"

"3"
Svi: System.Int32[]


In [233]:
let t1  = [|0; 2; 4; 6; 8; 10|]
t1[..2] |> printfn "%A"
t1[2] <- 25
t1[..2] |> printfn "%A"
t1[15]

[|0; 2; 4|]
[|0; 2; 25|]


Error: System.IndexOutOfRangeException: Index was outside the bounds of the array.
   at <StartupCode$FSI_0238>.$FSI_0238.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

In [234]:
let t1  = [|0; 2; 4; 6; 8; 10|]
t1 |> Array.append [|2;3|]

[ 2, 3, 0, 2, 4, 6, 8, 10 ]

In [235]:
t1 |> Array.fold (+) 0 |> printfn "%A"

30


## `seq` kontejner

In [236]:
let s1 : int seq  = [0; 2; 4; 6; 8; 10] |> Seq.ofList

In [237]:
let s2 = {0..2..10}

In [238]:
let s3 = seq { for i in 0..10 do 
                 if i % 2 = 0 then i 
             }

In [239]:
let s4 = Seq.init 6 (fun i -> i*2) 

In [240]:
s1 |> List.ofSeq |> printfn "%A"
s2 |> List.ofSeq |> printfn "%A"
s3 |> Seq.toList |> printfn "%A"
s4 |> List.ofSeq |> printfn "%A"

[0; 2; 4; 6; 8; 10]
[0; 2; 4; 6; 8; 10]
[0; 2; 4; 6; 8; 10]
[0; 2; 4; 6; 8; 10]


In [241]:
let s5 = seq { 
    yield 5
    yield 4
}
let s6 = seq {
    yield 3
    yield 2
    yield! s5
}
s6 |> printfn "%A"


seq [3; 2; 5; 4]


In [242]:
let s5 = seq {1 .. 10000000}
s5 |> Seq.iter (fun e -> if e = 25000 then printfn "foo" else ())


foo


In [243]:
let rand = System.Random()

let randomNumbers = 
  seq { while true do 
          yield rand.Next(100000) 
      }

let foo = 
  randomNumbers
  |> Seq.truncate 10
  |> Seq.sort
  |> Seq.toArray
  |> printfn "%A"


[|1411; 10942; 11080; 17915; 34845; 58296; 64388; 87020; 90268; 99309|]


In [244]:
open System
open System.Globalization

let rec fromDate (from : DateTime) = seq {
    yield from
    yield! (from.AddDays 1 |> fromDate)
}

let allDates = DateTime.MinValue |> fromDate

let groupDatesByWeekDays (someDate : DateTime) = 
  allDates
  |> Seq.skipWhile 
    (fun d -> d.Date < DateTime(someDate.Year, someDate.Month, 1))
  |> Seq.takeWhile
    (fun d -> d.Month = someDate.Month)
  |> Seq.groupBy 
    (fun (d : DateTime) -> d.ToString("ddd", CultureInfo("bs-BA")))

let printDay (someDate : DateTime) shouldInvertColor =
  let fc = ConsoleColor.White
  let bc = ConsoleColor.Black
  if shouldInvertColor then
    Console.ForegroundColor <- bc
    Console.BackgroundColor <- fc
    printf "%3d" someDate.Day
  else
    Console.ForegroundColor <- fc
    Console.BackgroundColor <- bc
    printf "%3d" someDate.Day
  Console.ResetColor()

let printEntireMonth (someDate : DateTime) =
  someDate.ToString("MMMM yyyy", CultureInfo("bs-BA"))  
  |> printfn "%19s" 
  seq {
    let groupedDates = groupDatesByWeekDays someDate 
    for (weekDay, wdDates) in groupedDates do
      weekDay |> printf "%s " 
      for day in wdDates do 
        if day = someDate then
          printDay day true
        else
          printDay day false
      printf "\n"
  } 

DateTime.Today |> printEntireMonth

      decembar 2023
pet   1  8 15 22 29
sub   2  9 16 23 30
ned   3 10 17 24 31
pon   4 11 18 25
uto   5 12 19 26
sri   6 13 20 27
čet   7 14 21 28


In [245]:
let altFromDate (from : DateTime) = seq {
    let mutable cDate = from
    while cDate < DateTime.MaxValue do
      yield cDate
      cDate <- cDate.AddDays(1)
}